In [1]:
pip install plotly dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 18.5 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [dash]
Note: you may need to restart the kernel to use updated packages.


In [5]:
# these are the UI comps, and libaries 
from dash import Dash, html, dcc, dash_table, callback, Input, Output, State
import pandas as pd
import plotly.express as px
import numpy as np
from pathlib import Path

# shows the program our csv we have aloocated
CSV_PATH = "gyro_data.csv" 

# loading our column and data and showing error if csv path is wrong
def load_gyro_csv(csv_path: str) -> pd.DataFrame:
    if not Path(csv_path).exists():
        raise FileNotFoundError(f":CSV wasn't found {csv_path}")

    df = pd.read_csv(csv_path)

    #detect timestamp col
    ts_candidates = ["timestamp_iso","timestamp","time","datetime","ts"]
    ts_col = next((c for c in ts_candidates if c in df.columns), None)
    if ts_col:
        df[ts_col] = pd.to_datetime(df[ts_col], errors="coerce")

    # Normalizes our data columns to gx, gy, gz
    rename_map_options = [
        {"gx_dps":"gx","gy_dps":"gy","gz_dps":"gz"},
        {"x":"gx","y":"gy","z":"gz"},
        {"gx":"gx","gy":"gy","gz":"gz"},
        {"gyro_x":"gx","gyro_y":"gy","gyro_z":"gz"},
        {"x_dps":"gx","y_dps":"gy","z_dps":"gz"},
    ]
    for mapping in rename_map_options:
        if all(src in df.columns for src in mapping.keys()):
            df = df.rename(columns=mapping)
            break

    # coerce to numeric, bad values become NaN
    for c in ["gx","gy","gz"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    # last checking to make sure we have our data before proceeding
    missing = [c for c in ["gx","gy","gz"] if c not in df.columns]
    if missing:
        raise ValueError(
            f"Could not find gyro columns. Need any of these schemas: "
            f"(gx_dps,gy_dps,gz_dps) or (x,y,z). Missing: {missing}"
        )

    # Dropping the rows with no gyro readings (NaN)
    df = df.dropna(subset=["gx","gy","gz"]).reset_index(drop=True)
    return df, ts_col

df, TS_COL = load_gyro_csv(CSV_PATH)

# UI layout
app = Dash(__name__)
app.title = "Gyroscope Dashboard"

GRAPH_TYPES = [
    {"label":"Line","value":"line"},
    {"label":"Scatter","value":"scatter"},
    {"label":"Histogram (distribution)","value":"hist"},
    {"label":"Box (distribution)","value":"box"},
    {"label":"Violin (distribution)","value":"violin"},
]
VAR_OPTIONS = [
    {"label":"X","value":"gx"},
    {"label":"Y","value":"gy"},
    {"label":"Z","value":"gz"},
]

def kpi_table(window_df: pd.DataFrame, vars_sel: list[str]) -> pd.DataFrame:
    # makes a compact summary
    stats = {}
    for v in vars_sel:
        s = window_df[v].dropna()
        if s.empty:
            continue
        stats[v] = {
            "count": int(s.count()),
            "mean":  float(s.mean()),
            "std":   float(s.std(ddof=1)) if s.count() > 1 else 0.0,
            "min":   float(s.min()),
            "median":float(s.median()),
            "max":   float(s.max()),
            "rms":   float(np.sqrt(np.mean(np.square(s)))),
        }
    return pd.DataFrame.from_dict(stats, orient="index").round(4).reset_index(names="variable")

app.layout = html.Div([
    html.H3("Gyroscope CSV - data handler"),
    html.Div([
        html.Div([
            html.Label("Chart type"),
            dcc.Dropdown(id="graph-type", options=GRAPH_TYPES, value="line", clearable=False),
        ], style={"width":"20%", "minWidth":"180px"}),

        html.Div([
            html.Label("Variables"),
            dcc.Dropdown(id="vars", options=VAR_OPTIONS,
                         value=["gx","gy","gz"], multi=True, clearable=False),
        ], style={"width":"30%", "minWidth":"240px"}),

        html.Div([
            html.Label("Samples per page (N)"),
            dcc.Input(id="num-samples", type="number", value=1000, min=10, step=10, debounce=True,
                      style={"width":"100%"}),
        ], style={"width":"18%", "minWidth":"160px"}),

        html.Div([
            html.Label("Navigation"),
            html.Div([
                html.Button("◀ Prev", id="prev-btn", n_clicks=0),
                html.Button("Next ▶", id="next-btn", n_clicks=0, style={"marginLeft":"8px"}),
            ]),
        ], style={"width":"20%", "minWidth":"180px"}),

    ], style={"display":"flex", "gap":"12px", "flexWrap":"wrap"}),

    # Store for window position (start index)
    dcc.Store(id="win-start", data=0),

    html.Hr(),

    dcc.Graph(id="gyro-graph"),

    html.H4("Data Summary"),
    dash_table.DataTable(
        id="summary",
        page_size=6,
        style_table={"overflowX":"auto"},
        style_cell={"textAlign":"center","padding":"6px"},
        style_header={"fontWeight":"bold"}
    ),

    html.H4("Current window, Raw rows"),
    dash_table.DataTable(
        id="raw",
        page_size=12,
        style_table={"overflowX":"auto"},
    ),
])

# updates by either using the nav window or the sample per page(N) 
@callback(
    Output("win-start","data"),
    Input("prev-btn","n_clicks"),
    Input("next-btn","n_clicks"),
    Input("num-samples","value"),
    State("win-start","data"),
    prevent_initial_call=False
)
def move_window(n_prev, n_next, n, start):
    total = len(df)
    n = int(n) if (n and n>0) else total
    # (next - prev) * n
    ctx = dash.ctx.triggered_id if hasattr(dash, "ctx") else None
    start = int(start or 0)

    if ctx == "next-btn":
        start += n
    elif ctx == "prev-btn":
        start -= n
    else:
        # num-samples changed or first load -> clamp start
        pass

    # Clamp
    start = max(0, min(start, max(0, total - n)))
    return start

# builds a figure for current tables n rows 
@callback(
    Output("gyro-graph","figure"),
    Output("summary","data"),
    Output("summary","columns"),
    Output("raw","data"),
    Output("raw","columns"),
    Input("graph-type","value"),
    Input("vars","value"),
    Input("num-samples","value"),
    Input("win-start","data"),
)
def update_view(gtype, vars_sel, n, start):
    vars_sel = [v for v in (vars_sel or []) if v in ["gx","gy","gz"]]
    if not vars_sel:
        vars_sel = ["gx","gy","gz"]

    total = len(df)
    n = int(n) if (n and n>0) else total
    start = int(start or 0)
    end = min(start + n, total)

    window = df.iloc[start:end].copy()

    # for x axis timestamp if we have one, use sample index if timestamp wasnt found
    if TS_COL and TS_COL in window.columns:
        xaxis = window[TS_COL]
        x_label = TS_COL
    else:
        xaxis = window.index
        x_label = "sample_index"

    # building our figure with the requested data respectivly 
    if gtype in ["line","scatter"]:
        # Long form for multiple traces keeping the webGL responsive
        plot_df = window[[*vars_sel]].copy()
        plot_df[x_label] = xaxis
        plot_long = plot_df.melt(id_vars=[x_label], value_vars=vars_sel,
                                 var_name="axis", value_name="value")
        if gtype == "line":
            fig = px.line(plot_long, x=x_label, y="value", color="axis",
                          render_mode="webgl" if len(plot_long)>2000 else "auto",
                          title=f"{gtype.title()} — {', '.join(vars_sel)}")
        else:
            fig = px.scatter(plot_long, x=x_label, y="value", color="axis",
                             render_mode="webgl" if len(plot_long)>2000 else "auto",
                             title=f"Scatter — {', '.join(vars_sel)}")

        fig.update_layout(xaxis_title=x_label, yaxis_title="dps (deg/sec)")
    elif gtype in ["hist","box","violin"]:
        # Distribution plots
        plot_long = window[vars_sel].copy().rename(columns={"gx":"X","gy":"Y","gz":"Z"})
        long = plot_long.melt(var_name="axis", value_name="value")
        if gtype == "hist":
            fig = px.histogram(long, x="value", color="axis", barmode="overlay",
                               nbins=60, opacity=0.65, title="Histogram")
            fig.update_layout(xaxis_title="dps (deg/sec)", yaxis_title="count")
        elif gtype == "box":
            fig = px.box(long, x="axis", y="value", points="outliers", title="Box plot")
            fig.update_layout(yaxis_title="dps (deg/sec)", xaxis_title="axis")
        else:
            fig = px.violin(long, x="axis", y="value", box=True, points="outliers",
                            title="Violin plot")
            fig.update_layout(yaxis_title="dps (deg/sec)", xaxis_title="axis")
    #fall back if it fails        
    else:
        fig = px.line(title="(unknown graph type)")

    # current table summary
    summary_df = kpi_table(window, vars_sel)
    sum_cols = [{"name": c, "id": c} for c in summary_df.columns]

    # shows timestamp raw table
    show_cols = [TS_COL] if TS_COL else []
    show_cols += vars_sel
    raw_df = window[show_cols].copy()
    raw_cols = [{"name": c, "id": c} for c in raw_df.columns]
    # returns the plot, summary 
    return fig, summary_df.to_dict("records"), sum_cols, raw_df.to_dict("records"), raw_cols

# runs in html format and not in jupyter
app.run(jupyter_mode="tab", debug=False)

Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>

[2025-08-30 22:03:56,644] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/arsalansharifizad/anaconda3/lib/python3.13/site-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/arsalansharifizad/anaconda3/lib/python3.13/site-packages/flask/app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/arsalansharifizad/anaconda3/lib/python3.13/site-packages/flask/app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/arsalansharifizad/anaconda3/lib/python3.13/site-packages/flask/app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^
  File "/Users/arsalansharifizad/anaconda3/lib/python3.13/site-packages/dash/dash.py", line 1494, in dispatch